In [1]:
from glob import glob

In [2]:
import sys
sys.path.append('../src/')

from core import CVMEME, extract_text_from_pdf, parse_json_scale
from prompts import cv_scale, scale, format_example

In [3]:
cvmeme = CVMEME()

In [6]:
cv_path = glob('../data/cv/*')[1]

In [7]:
cv_path

'../data/cv/AlSolCV.pdf'

In [8]:
cv = extract_text_from_pdf(cv_path, bytes=False)

In [7]:
cv, cv_processed = cvmeme.process_cv(cv_path, bytes=False)

TypeError: bad stream: type(stream)=<class 'str'>.

In [7]:
cv_scores = cvmeme.get_scores(cv_processed)

In [8]:
cv_scores

['пух_смокинге_шаблон', 'сверхразум', 'da_poebat_mne_gosling']

In [9]:
meme = cvmeme.rerank_memes(cv, cv_scores)

In [10]:
meme

'пух_смокинге_шаблон'

In [11]:
img = cvmeme.get_meme_img_path(meme)

In [12]:
img

'../data/meme/img/пух_смокинге_шаблон.jpg'

In [13]:
import telegram
telegram.__version__

'21.4'

In [4]:
cvmeme.get_other_respond("привет")

In [5]:
from random import random, choices

random()

0.3284822757205327

In [9]:
choices([1, 2, 3])

[1]

In [9]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [10]:
load_dotenv()

True

In [11]:
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],
    base_url=os.environ['LLM_HOST']
)

In [12]:
model_name = client.models.list().data[0].id

In [13]:
model_name

'google/gemma-2-2b-it'

In [14]:
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": cv_scale.format(
                    cv=cv,
                    scale=scale
                ) + format_example},
            ],
        }
    ],
    max_tokens=512,
    temperature=0.0,
)

# return response.choices[0].message.content

In [15]:
splitted = response.choices[0].message.content.split('\n')
splitted[0] == '```json' and splitted[-1] == '```', splitted[0], splitted[-1]

(False, '```json', '')

In [16]:
splitted[-2].strip()

'```'

In [17]:
parse_json_scale(response.choices[0].message.content)

return


ValueError: Invalid json format {'опыт_работы': 3, 'ответственность': 3, 'креативность': 3, 'коммуникабельность': 4, 'лидерские_качества': 2, 'стрессоустойчивость': 4, 'уровень_сложности_работы': 4, 'средняя_продолжительность_работы_на_одном_месте': 3, 'карьерный_рост': 3, 'уровень_дотошности_описания': 4, 'уровень_заумности': 3, 'уровень_высокомерия': 2, 'образование': 4, 'академичность': 3, 'иностранные_языки': [{'английский': 'хороший'}, {'итальянский': 'а2'}], 'профессиональные_сертификаты': []}

In [13]:
print(response.choices[0].message.content)

```json
{
    "опыт_работы": 3,
    "ответственность": 3,
    "креативность": 4,
    "коммуникабельность": 4,
    "лидерские_качества": 3,
    "стрессоустойчивость": 4,
    "уровень_сложности_работы": 4,
    "средняя_продолжительность_работы_на_одном_месте": 3,
    "карьерный_рост": 4,
    "уровень_дотошности_описания": 4,
    "уровень_заумности": 4,
    "уровень_высокомерия": 3,
    "образование": 4,
    "академичность": 4,
    "иностранные_языки": 4,
    "профессиональные_сертификаты": 2
}
``` 



In [17]:
base_prompt = """
Ты бот, который подбирает мем по резюме. 
Ответь на вопрос пользователя, если он на эту тему и попроси отправить pdf файл с резюме. 
НЕ ОТВЕЧАЙ на вопросы, которые не связаны с резюме.
Если тебя спросят, то мы не храним резюме пользователей.
Пиши ТОЛЬКО на русском языке.
"""

In [20]:
txt = "привет"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        # {
        #     "role": "system",
        #     "content": [
        #         {"type": "text", "text": base_prompt},
        #     ],
        # },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": base_prompt+'\n\nСообщение:\n'+txt},
            ],
        }
    ],
    max_tokens=512,
    temperature=0.3,
)

In [21]:
print(response.choices[0].message.content)

Привет! Я бот, который поможет тебе найти идеальный мем для твоего резюме. Чтобы я мог подобрать мем, мне нужно увидеть твое резюме. Пожалуйста, отправь мне pdf-файл с твоим резюме.


In [1]:
from PIL import Image

def resize_image_if_needed(image_path: str) -> str:
    max_size = (4096, 4096)
    with Image.open(image_path) as img:
        if img.width > max_size[0] or img.height > max_size[1]:
            img.thumbnail(max_size, Image.ANTIALIAS)
            resized_img_path = image_path.replace(".jpg", "_resized.jpg")
            img.save(resized_img_path, "JPEG")
            return resized_img_path
    return image_path

In [2]:
from glob import glob

In [3]:
images = glob('../data/meme/img/*')

In [4]:
max_size = (4096, 4096)

for image in images:
    with Image.open(image) as img:
        if img.width > max_size[0] or img.height > max_size[1]:
            print(image)

../data/meme/img/futurama_fry.jpg


In [5]:
import PIL
PIL.__version__

'10.2.0'

In [6]:
from datetime import datetime

In [7]:
interaction_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [8]:
interaction_time

'2024-08-03 01:09:57'

In [13]:
from random import choices, randint

In [11]:
choices([1, 2, 3], k=3, weights=[0.45, 0.35, 0.2])

[1, 2, 1]

In [18]:
randint(1, 3)

2